In [68]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from datetime import timedelta
from functools import reduce

In [54]:
df_1 = pd.read_excel('Table1.xlsx')
df_2 = pd.read_excel('Table2.xlsx')

In [55]:
df_2.head()

,LoanID,Region (English),Outstanding Balance,Additional Payment
0,499449,Adelaide,159.96,108
1,1560836,Algiers,1064.70,111
2,1565861,Athens,809.99,93
3,1580056,Adelaide,1163.89,188
4,1640416,Adelaide,265.58,157


In [56]:
df_1.head()

,Product Group,LoanID,Status,Borrower ID,Borrower DOB (MM/DD/YYYY),Region (Spanish),Cut Off Date,Origination Date,Origination Balance,Loan MaturityDate,Default Amount,Insurance
0,A,2946931,Defaulted,1842711,1978-03-01,Alejandría,2021-12-01,2018-09-20,6500.0,2023-09-01,124,N
1,A,2915187,Performing,281213,1973-09-01,Adelaida,2021-12-01,2018-08-22,9900.0,2026-09-01,0,N
2,A,2889592,Defaulted,1555948,1977-11-01,Arge,2021-12-01,2018-07-25,5600.0,2023-08-01,471,N
3,A,2752970,Performing,1595466,1961-01-01,Arge,2021-12-01,2018-03-19,5000.0,2022-03-01,0,Y
4,A,1883892,Performing,630149,1967-06-01,Alejandría,2021-12-01,2016-05-27,8200.0,2024-06-01,0,N


## ANS -1 

In [57]:
df_1['diff_in_days'] = df_1['Cut Off Date'] - df_1['Borrower DOB (MM/DD/YYYY)'] 
df_1['diff_in_years'] = df_1["diff_in_days"] / timedelta(days=365) 
avg_borrower_age = df_1.groupby('Product Group')['diff_in_years'].mean()
avg_borrower_age

Product Group
A    53.550466
B    52.262411
C    51.858904
Name: diff_in_years, dtype: float64

In [58]:
df_1['orig_year'] = df_1['Origination Date'].dt.year
origination_year = df_1.groupby('Product Group').agg({'orig_year':min})
origination_year

,orig_year
Product Group,
A,2015
B,2016
C,2011


In [59]:
total_accounts = df_1.groupby('Product Group').size().reset_index()
total_accounts.rename(columns={0:'Total Accounts'},inplace = True)
total_accounts

,Product Group,Total Accounts
0,A,100
1,B,100
2,C,100


In [60]:
df_3 = pd.merge(df_1,df_2,on='LoanID',how='inner')
total_balances = df_3.groupby('Product Group').agg({'Origination Balance':sum,'Outstanding Balance':sum})
total_balances

,Origination Balance,Outstanding Balance
Product Group,,
A,750700.0,317553.24
B,1087600.0,472023.35
C,365487.1,114248.06


In [61]:
innsured_loans = df_1.groupby('Product Group')['Insurance'].apply(lambda x: (x=='Y').sum()).reset_index(name='Insured Loans')
innsured_loans

,Product Group,Insured Loans
0,A,5
1,B,22
2,C,15


In [67]:
max_maturity_date = df_1.groupby('Product Group').agg({'Loan MaturityDate':max})
df_4  = pd.merge(max_maturity_date,df_1,on=['Product Group','Loan MaturityDate'],how='inner')
loan_id_maturity = df_4.drop_duplicates(subset = ['Product Group', 'Loan MaturityDate'], keep = 'first').reset_index(drop = True)
loanID_max_maturity = loan_id_maturity[['Product Group','LoanID']]
loanID_max_maturity

,Product Group,LoanID
0,A,2915187
1,B,2923352
2,C,2999641


## ANS -2 

In [79]:
df_test = [origination_year,innsured_loans,loanID_max_maturity,total_balances,total_accounts]
df_ans_2 = reduce(lambda  left,right: pd.merge(left,right,on=['Product Group'],how='inner'), df_test)
df_ans_2

,Product Group,orig_year,Insured Loans,LoanID,Origination Balance,Outstanding Balance,Total Accounts
0,A,2015,5,2915187,750700.0,317553.24,100
1,B,2016,22,2923352,1087600.0,472023.35,100
2,C,2011,15,2999641,365487.1,114248.06,100


## ANS -3 

In [80]:
max_originating_balance = df_1.groupby('Product Group').agg({'Origination Balance':max})
df_merged  = pd.merge(max_originating_balance,df_1,on=['Product Group','Origination Balance'],how='inner')
loan_id_originating_balance = df_merged.drop_duplicates(subset = ['Product Group', 'Origination Balance'], keep = 'first').reset_index(drop = True)
loanID_max_originating_balance = loan_id_originating_balance[['Product Group','LoanID']]
loanID_max_originating_balance

,Product Group,LoanID
0,A,2877789
1,B,2826001
2,C,2716547


## ANS -4

In [78]:
df_ques3 = pd.merge(df_1,df_2,on='LoanID',how='inner')
df_ans_3 = df_ques3.groupby(['Product Group']).apply(lambda x: x['Outstanding Balance'].sum()/x['Origination Balance'].sum()).reset_index(name='Balance Ammortized')
df_ans_3

,Product Group,Balance Ammortized
0,A,0.423010
1,B,0.434005
2,C,0.312591
